In [1]:
%matplotlib inline
from ase.build import bulk
from clusterx.parent_lattice import ParentLattice
from clusterx.super_cell import SuperCell
import numpy as np
from clusterx.visualization import juview
from clusterx.structures_set import StructuresSet
from clusterx.structure_selector import StructureSelector
from ase import Atoms
from clusterx.clusters.clusters_pool import ClustersPool
from clusterx.correlations import CorrelationsCalculator
from random import randint
from clusterx.calculators.emt import EMT2 
from clusterx.utils import parent_lattice_to_atat as plat2atat
from clusterx.utils import atat_to_cell as atat2plat
from clusterx.visualization import plot_property_vs_concentration
from clusterx.model import ModelBuilder
from clusterx.visualization import plot_optimization_vs_number_of_clusters
from clusterx.visualization import plot_predictions_vs_target
from sklearn.linear_model import LinearRegression
from ase.visualize import view

In [2]:
pri = bulk('Cs','bcc')
sub = bulk('Cl','bcc',a=1.085)

platt = ParentLattice(pri, substitutions=[sub])
plat2atat(platt,'lat_bcc.in')
scell = SuperCell(platt,[4,4,4])
platt.get_sublattice_types(pretty_print = True)


+--------------------------------------------------------------------+
|              The structure consists of 1 sublattices               |
+--------------------------------------------------------------------+
| Sublattice type |       Chemical symbols       |  Atomic numbers   |
+--------------------------------------------------------------------+
|        0        |         ['Cs' 'Cl']          |      [55 17]      |
+--------------------------------------------------------------------+



In [ ]:
# Collect 60 random structures in a StructuresSet object
sset = StructuresSet(platt)
nstruc = 60

for i in range(nstruc):
    concentration = {0:[randint(0,4*4*4)]}
    sset.add_structure(scell.gen_random(concentration))

In [ ]:
sset.get_images()[0]

In [ ]:
juview(sset,n=10)

In [ ]:
sset.set_calculator(EMT2())
sset.calculate_property("total_energy_emt")

In [ ]:
refs = StructuresSet(platt)
refs.add_structure(scell.gen_random({0:[0]})) # Pristine
refs.add_structure(scell.gen_random({0:[4*4*4]})) # Full Na substitution
refs.set_calculator(EMT2())
refs.calculate_property("total_energy_emt_refs")
ref_en = refs.get_property_values("total_energy_emt_refs")

In [ ]:
sset.write_files(formats=['lammps-data'])

In [ ]:
juview(SuperCell(platt,[2,2,2]))

In [28]:
a=0.5
pri = Atoms(positions=[[0,0,0]],cell=[[a,a,a],[a,-a,a],[a,a,-a]],pbc=(1,1,1)) # Atoms object. Only positions and cell are set
plat = ParentLattice(pri,site_symbols=[['Cs','Cl',]]) # ParentLattice object with assignement of species
#plat = atat2plat('lat.in')

r = 1.42 # This is large enough to include all clusters up to sqrt(2)*a
cpool = ClustersPool(plat, npoints=[0,1,2,2,3,4], radii=[0,0,1.0,0.866,1.0,1.0],)
#juview(cpool)

In [35]:
corrcal = CorrelationsCalculator("binary-linear", plat, cpool)
scell = SuperCell(plat,[4,4,4]) # Create a 4x4 supercell
structure = scell.gen_random({0:[5]}) # Random structure with 3 substitutional atoms
#juview(structure.get_atoms())

In [39]:
print(corrcal.get_binary_random_structure_correlations(0.25))

[1.         0.25       0.0625     0.0625     0.015625   0.00390625]


In [26]:
structure.get_fractional_concentrations()

{0: [0.921875, 0.078125]}

In [170]:
scell

SuperCell(symbols='Cs', pbc=True, cell=[[0.5, 0.5, 0.5], [0.5, -0.5, 0.5], [0.5, 0.5, -0.5]], tags=...)

In [6]:
cpool.display_info()


+-----------------------------------------------------------+
|                   Clusters Pool Info                      |
+-----------------------------------------------------------+
|       Index       |   Nr. of points   |      Radius       |
+-----------------------------------------------------------+
|         0         |         0         |       0.000       |
|         1         |         1         |       0.000       |
|         2         |         2         |       0.866       |
|         3         |         2         |       1.000       |
|         4         |         3         |       1.000       |
|         5         |         4         |       1.000       |
+-----------------------------------------------------------+



In [179]:
corrs = corrcal.get_cluster_correlations(structure) # Compute correlations for structure
orbl = corrcal.get_orbit_lengths(structure) # Get orbit lengths in structure M_\alpha

print("{0:<19s}|{1:<19s}|{2:<19s}".format("Cluster index","Orbit length","Correlation"))
for i in range(len(cpool)):
    print("{0:<19d}|{1:<19d}|{2:<19.5f}".format(i, orbl[i], corrs[i]))

Cluster index      |Orbit length       |Correlation        
0                  |1                  |1.00000            
1                  |8                  |0.00000            
2                  |16                 |0.00000            
3                  |12                 |0.00000            
4                  |48                 |0.00000            
5                  |24                 |0.00000            


In [6]:
cluster_orbit, cluster_multiplicity = cpool.get_cluster_orbit(cluster_index =5)

print("There are ",len(cluster_orbit),"symmetrically equivalent representations of the cluster in the supercell.")
print("The cluster multiplicity is ",cluster_multiplicity,".")

There are  162 symmetrically equivalent representations of the cluster in the supercell.
The cluster multiplicity is  6 .


In [161]:
juview(cluster_orbit,n=5)
cluster_orbit.write_clusters_db(db_name="temp.json")

In [162]:
temp = cluster_orbit.get_cpool_clusters()

In [163]:
temp[0]

In [165]:
temp[0].get_idxs()

[0, 1, 4, 9]

In [166]:
np.set_printoptions(suppress=True)
temp[0].get_positions()

array([[ 0. ,  0. ,  0. ],
       [ 0.5,  0.5, -0.5],
       [ 1. , -0. ,  0. ],
       [ 0.5,  0.5,  0.5]])

In [ ]:
# Collect 60 random structures in a StructuresSet object
sset = StructuresSet(plat)
nstruc = 5

for i in range(nstruc):
    concentration = {0:[randint(0,4*4)]}
    sset.add_structure(scell.gen_random(concentration))

In [ ]:
corrmat = corrcal.get_correlation_matrix(sset)

In [ ]:
cpool.display_info()

In [ ]:
corrmat

In [ ]:
corrs, orbl, (corrs*orbl)/np.sum(corrs*orbl)

In [ ]:
corrs, orbl, (corrs*orbl)/np.sum(corrs*orbl)

In [ ]:
mb = ModelBuilder(selector_type="linreg",selector_opts={'clusters_sets':'size'},estimator_type="skl_Lars",estimator_opts={"fit_intercept":False})
cemodel1 = mb.build(sset, cpool, "total_energy_emt") #Build CE model using the training data set
cpool_opt1 = mb.get_opt_cpool()

In [ ]:
cpool_opt1.display_info()

In [ ]:
cemodel1.report_errors(sset)
cpool_opt1.display_info(ecis=cemodel1.get_ecis())
cpool_opt1.write_clusters_db(db_name="cpool_opt.json")

In [ ]:
corrmat.shape, energies.shape

In [ ]:
model = LinearRegression(fit_intercept=False)
eci_mine = model.fit(corrmat,energies)

In [ ]:
eci_mine.coef_

In [ ]:
r_sq = model.score(corrmat,energies)

In [ ]:
r_sq

In [ ]:
cpool_opt1.display_info()